In [ ]:
pip install --upgrade pyLDAvis

In [ ]:
Australia_Policy = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/Australia2023EnergyPolicyReview_extracted_paragraphs.txt",
                    "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/AustraliaPublicPolicy.txt",
                    "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/Australia’s network of nuclear cooperation agreements.txt"
                   ]

France_Policy = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/Energy_Policy_France_2016_Review_extracted_paragraphs.txt"]


Singapore_Policy = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/singapore_nr_extracted_paragraphs.txt",
                    "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/SingaporePublicPolicy.txt"
                   ]


Australia_Sentiment_Nuclear = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_1.txt",
                               "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_2.txt",
                               "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_3.txt",
                               "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_4.txt",
                               ]

Australia_Sentiment_Solar = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_5.txt",
                             "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_public_opinion_6.txt",
                             ]


France_Sentiment_Nuclear = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/france_public_opinion_1.txt",
                            "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/france_public_opinion_3.txt",

                           ]
France_Sentiment_Solar = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/france_public_opinion_1.txt",
                          "/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/france_public_opinion_2.txt",
                         ]


Singapore_Sentiment_Nuclear = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/singapore_public_opinion_3.txt"]
Singapore_Sentiment_Solar =["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/singapore_public_opinion_2.txt"]




Australia_Stakeholder_Nuclear = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_stakeholder_info_3.txt"]
Australia_Stakeholder_Solar = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/aus_stakeholder_info_2.txt"]


France_Stakeholder = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/france_stakeholder_info_1.txt"]

Singapore_Stakeholder = ["/data/drive/MyDrive/Colab Notebooks/Ali -Australia/Data/WebScrappedFiles/singapore_stakeholder_info_1.txt"]


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

# Save Word Cloud as Image
def create_wordcloud(policy_files, title, filename):
    policy_text = load_text(policy_files)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(policy_text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.savefig(filename, format='png')
    plt.close()

# Save Barplot as Image
def sentiment_barplot(data, filename):
    df_sentiment = pd.DataFrame(data, columns=['Country', 'Energy Type', 'Sentiment'])
    sns.barplot(x='Country', y='Sentiment', hue='Energy Type', data=df_sentiment)
    plt.title('Sentiment Analysis by Energy Type and Country')
    plt.savefig(filename, format='png')
    plt.close()

# Save Scatter Plot as Image
def scatter_plot_policy_impact(df_policy_impact, title, filename):
    plt.figure(figsize=(12, 8))
    plt.scatter(df_policy_impact['TF-IDF Score'], df_policy_impact['Average Sentiment'], s=100, c=df_policy_impact['TF-IDF Score'], cmap='viridis', alpha=0.7)
    for i in range(len(df_policy_impact)):
        plt.text(df_policy_impact['TF-IDF Score'][i], df_policy_impact['Average Sentiment'][i], df_policy_impact['Word'][i], fontsize=9)
    plt.title(title)
    plt.xlabel('Policy Topic Prominence (TF-IDF Score)')
    plt.ylabel('Average Sentiment')
    plt.colorbar(label='TF-IDF Score')
    plt.grid(True)
    plt.savefig(filename, format='png')
    plt.close()

# Save Heatmap as Image
def document_similarity_heatmap(country_policy_files, countries, energy_type, filename):
    policy_texts = []
    doc_labels = []

    for country, files in country_policy_files.items():
        for file in files:
            with open(file, 'r', encoding='utf-8') as f:
                policy_texts.append(f.read())
                doc_labels.append(f"{country} - {energy_type}")

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(policy_texts)
    cosine_sim = cosine_similarity(tfidf_matrix)

    df_sim = pd.DataFrame(cosine_sim, index=doc_labels, columns=doc_labels)

    plt.figure(figsize=(12, 10))
    sns.heatmap(df_sim, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(f'Document Similarity Heatmap ({energy_type.capitalize()})')
    plt.savefig(filename, format='png')
    plt.close()

# Save Radar Chart and Interactive Plotly Charts as HTML
def save_plotly_chart(fig, filename):
    pio.write_html(fig, file=filename, auto_open=False)

# Example usage for generating and saving the visualizations
create_wordcloud(Australia_Policy, 'Word Cloud for Australia Energy Policies', 'australia_wordcloud.png')
create_wordcloud(France_Policy, 'Word Cloud for France Energy Policies', 'france_wordcloud.png')
create_wordcloud(Singapore_Policy, 'Word Cloud for Singapore Energy Policies', 'singapore_wordcloud.png')

data = [
    ('Australia', 'Nuclear', analyze_sentiment(Australia_Sentiment_Nuclear)),
    ('Australia', 'Solar', analyze_sentiment(Australia_Sentiment_Solar)),
    ('France', 'Nuclear', analyze_sentiment(France_Sentiment_Nuclear)),
    ('France', 'Solar', analyze_sentiment(France_Sentiment_Solar)),
    ('Singapore', 'Nuclear', analyze_sentiment(Singapore_Sentiment_Nuclear)),
    ('Singapore', 'Solar', analyze_sentiment(Singapore_Sentiment_Solar))
]
sentiment_barplot(data, 'sentiment_barplot.png')

policy_impact, avg_sentiment = policy_impact_analysis(Australia_Policy, Australia_Sentiment_Nuclear)
df_policy_impact = pd.DataFrame(policy_impact, columns=['Word', 'TF-IDF Score', 'Average Sentiment'])
scatter_plot_policy_impact(df_policy_impact, 'Policy Impact on Public Opinion (Australia Nuclear)', 'australia_policy_impact.png')

country_policy_files = {'Australia': Australia_Policy, 'France': France_Policy, 'Singapore': Singapore_Policy}
document_similarity_heatmap(country_policy_files, countries=['Australia', 'France', 'Singapore'], energy_type='Nuclear', filename='nuclear_heatmap.png')

# Plotly interactive plots
df_scatter = create_scatter_plot_df(France_Policy, Australia_Sentiment_Nuclear, 'Australia', 'Nuclear')
fig_scatter = px.scatter(df_scatter, x='TF-IDF Score', y='Average Sentiment', color='Average Sentiment',
                 hover_data=['Topic'], title='Policy Impact on Public Opinion (Australia Nuclear)',
                 labels={'TF-IDF Score': 'Policy Topic Prominence (TF-IDF Score)', 'Average Sentiment': 'Average Sentiment'},
                 template='plotly_white')
save_plotly_chart(fig_scatter, 'scatter_plot.html')

df_radar = create_radar_chart_df(Australia_Policy, 'Australia', 'Nuclear')
fig_radar = go.Figure()

fig_radar.add_trace(go.Scatterpolar(
      r=df_radar['TF-IDF Score'],
      theta=df_radar['Topic'],
      fill='toself',
      name='Australia Nuclear'
))

fig_radar.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, max(df_radar['TF-IDF Score'])]
    )),
  showlegend=False,
  title='Radar Chart for Stakeholder Analysis (Australia Nuclear)'
)
save_plotly_chart(fig_radar, 'radar_chart.html')


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

def create_scatter_plot_df(policy_files, sentiment_files, country, energy_type):
    tfidf = TfidfVectorizer(stop_words='english')
    policy_text = [open(file, 'r', encoding='utf-8').read() for file in policy_files]
    tfidf_matrix = tfidf.fit_transform(policy_text)
    tfidf_scores = tfidf_matrix.sum(axis=0).A1
    words = tfidf.get_feature_names_out()

    sentiments = []
    for file in sentiment_files:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            sentiment = TextBlob(text).sentiment.polarity
            sentiments.append(sentiment)

    avg_sentiment = np.mean(sentiments)

    data = {
        'Country': [country] * len(words),
        'Energy Type': [energy_type] * len(words),
        'Topic': words,
        'TF-IDF Score': tfidf_scores,
        'Average Sentiment': avg_sentiment
    }

    return pd.DataFrame(data)

# File paths for policy and sentiment documents
policy_files_map = {
    'Australia': {'Nuclear': Australia_Policy, 'Solar': Australia_Policy},  # Using the same policy files for both
    'France': {'Nuclear': France_Policy, 'Solar': France_Policy},            # Using the same policy files for both
    'Singapore': {'Nuclear': Singapore_Policy, 'Solar': Singapore_Policy}    # Using the same policy files for both
}

sentiment_files_map = {
    'Australia': {'Nuclear': Australia_Sentiment_Nuclear, 'Solar': Australia_Sentiment_Solar},
    'France': {'Nuclear': France_Sentiment_Nuclear, 'Solar': France_Sentiment_Solar},
    'Singapore': {'Nuclear': Singapore_Sentiment_Nuclear, 'Solar': Singapore_Sentiment_Solar}
}

# Create data frames for each country and energy type
dfs = []
for country in policy_files_map:
    for energy_type in policy_files_map[country]:
        df = create_scatter_plot_df(policy_files_map[country][energy_type], sentiment_files_map[country][energy_type], country, energy_type)
        dfs.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Create the initial scatter plot for the first country and energy type
fig = px.scatter(combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Nuclear'],
                 x='TF-IDF Score', y='Average Sentiment', color='Average Sentiment',
                 hover_data=['Topic'], title='Policy Impact on Public Opinion By Country andn Energy Type',
                 labels={'TF-IDF Score': 'Policy Topic Prominence (TF-IDF Score)', 'Average Sentiment': 'Average Sentiment'},
                 template='plotly_white')

# Update traces with marker size and line properties
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))

# Add dropdown menus for filtering
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Nuclear']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Nuclear']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']}],
                    label="Australia - Nuclear",
                    method="update"
                ),
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Solar']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Solar']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'Australia'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']}],
                    label="Australia - Solar",
                    method="update"
                ),
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Nuclear']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Nuclear']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']}],
                    label="France - Nuclear",
                    method="update"
                ),
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Solar']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Solar']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'France'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']}],
                    label="France - Solar",
                    method="update"
                ),
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Nuclear']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Nuclear']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Nuclear']['Average Sentiment']}],
                    label="Singapore - Nuclear",
                    method="update"
                ),
                dict(
                    args=[{'x': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Solar']['TF-IDF Score']],
                           'y': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']],
                           'text': [combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Solar']['Topic']],
                           'marker.color': combined_df[combined_df['Country'] == 'Singapore'][combined_df['Energy Type'] == 'Solar']['Average Sentiment']}],
                    label="Singapore - Solar",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.17,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Show the interactive plot with dropdown filters
fig.show()

In [ ]:
import plotly.io as pio

# Assuming fig is the Plotly figure created in the previous Python code
pio.write_html(fig, file='interactive_scatter_plot.html', auto_open=False)

In [ ]:
import spacy
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from textblob import TextBlob

# Load the pre-trained spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Combine all relevant sentiment documents into a dictionary
sentiment_docs = {
    'Australia_Nuclear': Australia_Sentiment_Nuclear,
    'Australia_Solar': Australia_Sentiment_Solar,
    'France_Nuclear': France_Sentiment_Nuclear,
    'France_Solar': France_Sentiment_Solar,
    'Singapore_Nuclear': Singapore_Sentiment_Nuclear,
    'Singapore_Solar': Singapore_Sentiment_Solar
}

# Combine all stakeholder-related documents into a dictionary
stakeholder_docs = {
    'Australia_Stakeholder_Nuclear': Australia_Stakeholder_Nuclear,
    'Australia_Stakeholder_Solar': Australia_Stakeholder_Solar,
    'France_Stakeholder': France_Stakeholder,
    'Singapore_Stakeholder': Singapore_Stakeholder
}

# Function to calculate sentiment
def calculate_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Perform sentiment analysis for each sentiment document
sentiment_scores = {}
for sentiment_type, files in sentiment_docs.items():
    combined_text = ""
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            combined_text += f.read()
    sentiment = calculate_sentiment(combined_text)
    sentiment_scores[sentiment_type] = sentiment

# Extract mentions of stakeholders in the public sentiment documents
stakeholder_mentions = defaultdict(lambda: defaultdict(int))

for sentiment_type, files in sentiment_docs.items():
    combined_text = ""
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            combined_text += f.read()

    doc = nlp(combined_text)

    # Count the mentions of stakeholder entities (organizations)
    for ent in doc.ents:
        if ent.label_ == "ORG":  # Organization
            for stakeholder_type, stakeholder_files in stakeholder_docs.items():
                for stakeholder_file in stakeholder_files:
                    with open(stakeholder_file, 'r', encoding='utf-8') as sf:
                        if ent.text in sf.read():
                            stakeholder_mentions[stakeholder_type][sentiment_type] += 1

# Organize data for radar plot
radar_data = []
for stakeholder_type, mentions in stakeholder_mentions.items():
    data_row = {'Stakeholder_Type': stakeholder_type}
    for sentiment_type, score in sentiment_scores.items():
        data_row[sentiment_type] = mentions.get(sentiment_type, 0)
    radar_data.append(data_row)

# Convert to DataFrame
df_radar = pd.DataFrame(radar_data)

# Create radar plot
fig = go.Figure()

# Add each stakeholder to the radar plot
for i in range(len(df_radar)):
    fig.add_trace(go.Scatterpolar(
        r=df_radar.iloc[i, 1:],  # Select all columns except the first one which is 'Stakeholder_Type'
        theta=df_radar.columns[1:],  # Use the column names as the theta values
        fill='toself',
        name=df_radar['Stakeholder_Type'][i]  # Name the trace based on the stakeholder type
    ))

# Customize layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, df_radar.drop(columns='Stakeholder_Type').max().max()]  # Adjust range based on data
        )),
    showlegend=True,
    title='Public Sentiment vs. Stakeholder Mentions (Radar Plot)'
)

# Save the radar plot as an HTML file
fig.write_html("radar_plot_stakeholders.html")